In [4]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2
%reload_ext tensorboard
import os
import torch
import torchvision
import torchvision.transforms.functional as fn
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from models.models import LPCNNv1
from models.lpcnnv2 import LPCNNv2
from src.data.car_plate_dataset import UFPRDataset, id_to_filepath, annotate_frame_with_bb

In [ ]:
os.chdir(r"C:\Users\Arya\workspace\ProjectSentry\data\raw\UFPR-ALPR dataset\training")
%tensorboard --logdir=lightning_logs/

In [8]:
train_dataset = UFPRDataset(
    "C:\\Users\\Arya\\workspace\\ProjectSentry\\data\\raw\\UFPR-ALPR dataset\\training",
    resize=torchvision.transforms.Resize(size=500),
    # grayscale=torchvision.transforms.Grayscale()
)
# val_dataset = UFPRDataset(
#     "C:\\Users\\Arya\\workspace\\ProjectSentry\\data\\raw\\UFPR-ALPR dataset\\testing",
#     resize=torchvision.transforms.Resize(size=500),
#     grayscale=torchvision.transforms.Grayscale()
# )

finished setting up data


In [99]:
# train_dataset.labels.items()
# train_dataset.__getitem__(29)
# train_dataset.labels["000130"][1]

array([1152,  506, 1209,  531])

In [8]:
a = torch.rand((1, 500, 888, 3))
print(a.shape)
print(a.transpose(1, 3).shape)

torch.Size([1, 500, 888, 3])
torch.Size([1, 3, 888, 500])


In [16]:
train_dataloader = DataLoader(train_dataset, batch_size=32, num_workers=8)
logger = TensorBoardLogger('lightning_logs')
trainer = pl.Trainer(max_epochs=4, limit_train_batches=0.25, logger=logger)
model = LPCNNv2()

trainer.fit(model, train_dataloaders=train_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Using cache found in C:\Users\Arya/.cache\torch\hub\pytorch_vision_v0.10.0

  | Name         | Type              | Params
---------------------------------------------------
0 | features1    | Sequential        | 1.3 M 
1 | features2    | Sequential        | 19.9 M
2 | rel1         | ReLU              | 0     
3 | pool         | AdaptiveAvgPool2d | 0     
4 | bounding_box | Sequential        | 3.1 K 
---------------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.151    Total estimated model params size (MB)
C:\Users\Arya\anaconda3\envs\vision\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you w

Training: 0it [00:00, ?it/s]

input: torch.Size([64, 3, 500, 888])


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 454656000 bytes.

In [ ]:
model = LPCNNv1.load_from_checkpoint(r"C:\Users\Arya\Downloads\epoch=11-step=21600.ckpt")

model.eval()
# val_dataloader = DataLoader(val_dataset, num_workers=8)


In [ ]:
# trainer = pl.Trainer()
# trainer.validate(model=model, dataloaders=val_dataloader)

In [37]:
# Testing one output from the trained model
plt.close()
photo_set = "0015"
num = "03"
image_id = f"track{photo_set}[{num}]"
os.chdir(fr"C:/Users/Arya/workspace/ProjectSentry/data/raw/UFPR-ALPR dataset/training/track{photo_set}")
image = Image.open(f"{image_id}.png" )
bb_model = model
fig, ax = annotate_frame_with_bb(
    image,
    image_id,
    bb_model,
    torchvision.transforms.Resize(size=500),
    torchvision.transforms.Grayscale()
)
plt.show()

C:\Users\Arya\AppData\Local\Temp\ipykernel_20968\3465078491.py:2: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.close()


<IPython.core.display.Javascript object>

True (blue): [490 228 533 245]
Model predicted (red): tensor([374.3509, 163.5187, 419.4533, 180.5947])
